In [1]:
import os
import pandas as pd
import json
import matchms
from pathlib import Path
import py4cytoscape as p4c
from zipfile import ZipFile
import io
from IPython.display import SVG, display
from extractor.network import Network
from extractor.compounds import Compounds
from extractor.manufactured.datadirs import *
os.chdir("../")
os.getcwd()

'/home/olivier/Local/Nextcloud LAMSADE/Recherche/AMCD/Chimie/MS2DECIDE/Data/Extractor'

In [8]:
compounds = pd.read_csv(GENERATED_DIR_TABLES / "Compounds joined.tsv", sep="\t").set_index("Id")
cols_indicators = ["cg", "cs", "ci", "tgs", "tgi", "tsi", "Rank K"]
cols_ranks = compounds.columns.str.startswith("Rank")
compounds

,Chemical name,Chemical class,InChI,Relative molecular weight,Precursor m/z GNPS,Precursor m/z Sirius,Retention time (sec) GNPS,Retention time (sec) Sirius,Precursor m/z GNPS − relative molecular weight,Analog InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.02,...,Rank GNPS iterated,Rank min Sirius,Rank max Sirius,Ranks Sirius,Rank min ISDB-LOTUS,Rank max ISDB-LOTUS,Ranks ISDB-LOTUS,Rank min K,Rank max K,Rank K
Id,,,,,,,,,,,,,,,,,,,,,
1,13-hydroxylupanine,Bisquinolizidine alkaloid,InChI=1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14...,264.369,265.1918,265.19177,37.668,37.67,0.8228,InChI=1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14...,...,52,64,64,64,80,80,80,60,60,60
2,3-beta-amino-pregnane,Steroidal alkaloid,InChI=1S/C21H37N/c1-4-14-6-8-18-17-7-5-15-13-1...,303.534,304.3004,304.30036,169.694,169.69,0.7664,NaN,...,33,8,8,8,31,31,31,24,24,24
3,3-beta-dimethylamino-20-beta-hydroxy-5-pregnene,Steroidal alkaloid,InChI=1S/C23H39NO/c1-15(25)19-8-9-20-18-7-6-16...,345.571,346.3101,346.31010,222.932,222.93,0.7391,InChI=1S/C19H36O4/c1-3-4-5-6-7-8-9-10-11-12-13...,...,16,4,4,4,35,35,35,11,11,11
4,3-beta-methylamino-20-oxo-5-pregnene,Steroidal alkaloid,InChI=1S/C22H35NO/c1-14(24)18-7-8-19-17-6-5-15...,329.528,330.2792,330.27918,128.041,128.04,0.7512,,...,24,31,31,31,38,38,38,27,27,27
5,Alchorneine,Imidazopyrimidine alkaloid,"InChI=1S/C12H19N3O/c1-9(2)10-8-14-7-6-12(3,4)1...",221.304,222.1599,222.15989,92.029,92.03,0.8559,NaN,...,21,2,3,2–3,24,24,24,12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Unreported 525,MIA,NaN,NaN,525.2782,525.27808,1113.980,1113.98,NaN,NaN,...,13,1,1,1,1,5,1–5,6,6,6
92,Unreported 645,MIA,NaN,NaN,645.3782,645.37816,1489.705,1489.70,NaN,InChI=1S/C35H58O9/c1-19(2)32-24(7)27(36)18-35(...,...,8,38,38,38,18,18,18,9,9,9
93,Unreported 647,MIA,NaN,NaN,647.3246,647.39583,1286.570,1340.16,NaN,InChI=1S/C38H46O9/c1-21(2)11-10-18-36(8)19-17-...,...,1,24,24,24,9,9,9,1,1,1


In [12]:
compounds.loc[:, cols_indicators]
# Change parameters to see all lines
with pd.option_context('display.max_rows', None):
    display(compounds.loc[:, cols_indicators].sort_values("tgs"))

,cg,cs,ci,tgs,tgi,tsi,Rank K
Id,,,,,,,
20,0.533312,0.858,0.254859,0.000000,0.000000,0.155844,46
30,0.138939,0.460,0.480256,0.022727,0.000000,0.215686,17
18,0.186500,0.511,0.144766,0.027397,0.132075,0.059701,22
58,0.304558,0.644,0.447631,0.042254,0.042254,1.000000,51
15,0.240888,0.145,0.428177,0.060870,0.120000,0.123810,14
50,0.129438,0.269,0.316319,0.066667,0.063492,0.203125,7
93,0.041077,0.200,0.016130,0.071429,0.080882,0.097222,1
3,0.256391,0.034,0.306876,0.072464,0.038961,0.253731,11
77,0.435033,0.125,0.137701,0.078125,0.104167,0.068493,29


In [15]:
compounds.loc[90:95, cols_ranks]

,Rank min GNPS original,Rank max GNPS original,Ranks GNPS original,Rank min GNPS iterated,Rank max GNPS iterated,Rank GNPS iterated,Rank min Sirius,Rank max Sirius,Ranks Sirius,Rank min ISDB-LOTUS,Rank max ISDB-LOTUS,Ranks ISDB-LOTUS,Rank min K,Rank max K,Rank K
Id,,,,,,,,,,,,,,,
90,9,9,9,7,7,7,30,30,30,10,10,10,5,5,5
91,15,15,15,13,13,13,1,1,1,1,5,1–5,6,6,6
92,10,10,10,8,8,8,38,38,38,18,18,18,9,9,9
93,3,3,3,1,1,1,24,24,24,9,9,9,1,1,1
94,1,2,1–2,47,47,47,17,17,17,1,5,1–5,35,35,35
95,5,5,5,3,3,3,12,12,12,12,12,12,4,4,4
